# Toistuvat neuroverkot

Edellisessä moduulissa käytimme tekstin semanttisesti rikkaita esityksiä ja yksinkertaista lineaarista luokitinta upotusten päällä. Tämä arkkitehtuuri pystyy vangitsemaan sanojen yhdistetyn merkityksen lauseessa, mutta se ei ota huomioon sanojen **järjestystä**, koska upotusten päälle tehty yhdistämisoperaatio poistaa tämän tiedon alkuperäisestä tekstistä. Koska nämä mallit eivät pysty mallintamaan sanojen järjestystä, ne eivät voi ratkaista monimutkaisempia tai epäselvempiä tehtäviä, kuten tekstin generointia tai kysymyksiin vastaamista.

Jotta voisimme ymmärtää tekstisekvenssin merkityksen, meidän täytyy käyttää toista neuroverkkoarkkitehtuuria, jota kutsutaan **toistuvaksi neuroverkoksi** eli RNN:ksi. RNN:ssä syötämme lauseen verkon läpi yksi symboli kerrallaan, ja verkko tuottaa jonkin **tilan**, jonka syötämme verkkoon uudelleen seuraavan symbolin kanssa.

Annettua syötesequenssia $X_0,\dots,X_n$ käyttäen RNN luo neuroverkkolohkojen sekvenssin ja opettaa tämän sekvenssin päästä päähän takaisinlevityksen avulla. Jokainen verkkolohko ottaa syötteenä parin $(X_i,S_i)$ ja tuottaa tuloksena $S_{i+1}$. Lopullinen tila $S_n$ tai ulostulo $X_n$ syötetään lineaariseen luokittimeen tuloksen tuottamiseksi. Kaikilla verkkolohkoilla on samat painot, ja ne opetetaan päästä päähän yhdellä takaisinlevityskierroksella.

Koska tilavektorit $S_0,\dots,S_n$ kulkevat verkon läpi, se pystyy oppimaan sanojen väliset sekventiaaliset riippuvuudet. Esimerkiksi, kun sana *not* esiintyy jossain sekvenssissä, se voi oppia kumoamaan tiettyjä elementtejä tilavektorissa, mikä johtaa negaatioon.

> Koska kaikkien RNN-lohkojen painot kuvassa ovat jaettuja, sama kuva voidaan esittää yhtenä lohkona (oikealla) toistuvalla palautesilmukalla, joka syöttää verkon ulostulotilan takaisin syötteeseen.

Katsotaanpa, kuinka toistuvat neuroverkot voivat auttaa meitä luokittelemaan uutisaineistomme.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Yksinkertainen RNN-luokitin

Yksinkertaisessa RNN:ssä jokainen rekursiivinen yksikkö on yksinkertainen lineaarinen verkko, joka ottaa yhdistetyn syötevektorin ja tilavektorin ja tuottaa uuden tilavektorin. PyTorch edustaa tätä yksikköä `RNNCell`-luokalla, ja tällaisten solujen verkkoa - `RNN`-kerroksena.

RNN-luokittimen määrittämiseksi käytämme ensin upotuskerrosta syötesanaston ulottuvuuden pienentämiseksi ja sen jälkeen RNN-kerrosta sen päällä:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Huom:** Käytämme tässä yksinkertaisuuden vuoksi kouluttamatonta upotuskerrosta, mutta vielä parempien tulosten saavuttamiseksi voimme käyttää valmiiksi koulutettua upotuskerrosta, kuten Word2Vec- tai GloVe-upotuksia, jotka on kuvattu edellisessä osiossa. Ymmärtämisen parantamiseksi kannattaa mukauttaa tämä koodi toimimaan valmiiksi koulutettujen upotusten kanssa.

Tässä tapauksessa käytämme täydennettyä datalaturia, joten jokaisessa erässä on sama määrä täydennettyjä sekvenssejä, jotka ovat samanpituisia. RNN-kerros ottaa upotustensoreiden sekvenssin ja tuottaa kaksi ulostuloa:
* $x$ on RNN-solujen ulostulojen sekvenssi jokaisessa vaiheessa
* $h$ on viimeisen sekvenssin elementin lopullinen piilotettu tila

Sen jälkeen sovellamme täysin yhdistettyä lineaarista luokittelijaa saadaksemme luokkien määrän.

> **Huom:** RNN:t ovat melko vaikeita kouluttaa, koska kun RNN-solut puretaan sekvenssin pituuden mukaan, takaisinkuljetuksessa mukana olevien kerrosten määrä kasvaa huomattavasti. Siksi meidän täytyy valita pieni oppimisnopeus ja kouluttaa verkkoa suuremmalla datamäärällä hyvien tulosten saavuttamiseksi. Tämä voi viedä melko paljon aikaa, joten GPU:n käyttö on suositeltavaa.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Pitkäkestoiset lyhytmuistiyksiköt (LSTM)

Yksi klassisten RNN-verkkojen suurimmista ongelmista on niin sanottu **häviävien gradienttien** ongelma. Koska RNN-verkkoja koulutetaan päästä päähän yhdessä takaisinlevitysvaiheessa, niillä on vaikeuksia välittää virhettä verkon ensimmäisille kerroksille, minkä vuoksi verkko ei pysty oppimaan suhteita kaukaisten tokenien välillä. Yksi tapa välttää tämä ongelma on ottaa käyttöön **eksplisiittinen tilanhallinta** käyttämällä niin sanottuja **portteja**. Tämän tyyppisistä arkkitehtuureista tunnetuimmat ovat **pitkäkestoiset lyhytmuistiyksiköt** (LSTM) ja **porttiohjattu reläyksikkö** (GRU).

![Kuva, joka näyttää esimerkin pitkäkestoisesta lyhytmuistiyksiköstä](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM-verkko on järjestetty samalla tavalla kuin RNN, mutta siinä on kaksi tilaa, jotka siirtyvät kerrokselta toiselle: varsinainen tila $c$ ja piilovektori $h$. Jokaisessa yksikössä piilovektori $h_i$ yhdistetään syötteeseen $x_i$, ja ne ohjaavat, mitä tilalle $c$ tapahtuu **porttien** kautta. Jokainen portti on hermoverkko, jossa on sigmoid-aktivaatio (tuotos alueella $[0,1]$), ja sitä voidaan ajatella bittimaskina, kun se kerrotaan tilavektorilla. Kuvan yllä vasemmalta oikealle portit ovat seuraavat:
* **Unohtamisportti** ottaa piilovektorin ja määrittää, mitkä vektorin $c$ komponentit tulee unohtaa ja mitkä päästää läpi.
* **Syöteportti** ottaa tietoa syötteestä ja piilovektorista ja lisää sen tilaan.
* **Ulostuloportti** muuntaa tilan jonkin lineaarisen kerroksen kautta, jossa on $\tanh$-aktivaatio, ja valitsee sitten joitakin sen komponentteja piilovektorin $h_i$ avulla tuottaakseen uuden tilan $c_{i+1}$.

Tilan $c$ komponentteja voidaan ajatella eräänlaisina lippuina, joita voidaan kytkeä päälle ja pois päältä. Esimerkiksi, kun kohtaamme sekvenssissä nimen *Alice*, voimme olettaa, että se viittaa naispuoliseen hahmoon, ja nostaa tilassa lipun, joka ilmaisee, että lauseessa on naispuolinen substantiivi. Kun myöhemmin kohtaamme ilmauksen *and Tom*, nostamme lipun, joka ilmaisee, että kyseessä on monikollinen substantiivi. Näin manipuloimalla tilaa voimme mahdollisesti seurata lauseen osien kieliopillisia ominaisuuksia.

> **Note**: Erinomainen resurssi LSTM:n sisäisen rakenteen ymmärtämiseen on Christopher Olahin loistava artikkeli [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/).

Vaikka LSTM-solun sisäinen rakenne saattaa näyttää monimutkaiselta, PyTorch piilottaa tämän toteutuksen `LSTMCell`-luokan sisälle ja tarjoaa `LSTM`-objektin koko LSTM-kerroksen esittämiseen. Näin ollen LSTM-luokittelijan toteutus on melko samanlainen kuin yksinkertaisen RNN:n, jonka näimme aiemmin:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Pakatut sekvenssit

Esimerkissämme jouduimme täyttämään kaikki minibatchin sekvenssit nollavektoreilla. Vaikka tämä aiheuttaa jonkin verran muistin hukkaa, RNN:ien kohdalla on vielä kriittisempää, että täytetyt syötteet luovat ylimääräisiä RNN-soluja, jotka osallistuvat koulutukseen, mutta eivät sisällä mitään merkittävää syötetietoa. Olisi paljon parempi kouluttaa RNN vain todellisen sekvenssin pituuden mukaan.

Tätä varten PyTorchissa on otettu käyttöön erityinen täytettyjen sekvenssien tallennusmuoto. Oletetaan, että meillä on täytetty minibatch, joka näyttää tältä:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Tässä 0 edustaa täytettyjä arvoja, ja syötteen sekvenssien todellinen pituusvektori on `[5,3,1]`.

Jotta RNN voidaan kouluttaa tehokkaasti täytetyillä sekvensseillä, haluamme aloittaa ensimmäisen RNN-soluryhmän koulutuksen suurella minibatchilla (`[1,6,9]`), mutta sitten lopettaa kolmannen sekvenssin käsittelyn ja jatkaa lyhennetyillä minibatcheilla (`[2,7]`, `[3,8]`) ja niin edelleen. Näin ollen pakattu sekvenssi esitetään yhtenä vektorina - tässä tapauksessa `[1,6,9,2,7,3,8,4,5]`, ja pituusvektorina (`[5,3,1]`), jonka avulla alkuperäinen täytetty minibatch voidaan helposti rekonstruoida.

Pakattujen sekvenssien tuottamiseen voidaan käyttää `torch.nn.utils.rnn.pack_padded_sequence`-funktiota. Kaikki rekursiiviset kerrokset, mukaan lukien RNN, LSTM ja GRU, tukevat pakattuja sekvenssejä syötteenä ja tuottavat pakatun ulostulon, joka voidaan purkaa käyttämällä `torch.nn.utils.rnn.pad_packed_sequence`.

Jotta voimme tuottaa pakatun sekvenssin, meidän täytyy välittää pituusvektori verkolle, ja siksi tarvitsemme erilaisen funktion minibatchien valmisteluun:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Todellinen verkko olisi hyvin samanlainen kuin yllä oleva `LSTMClassifier`, mutta `forward`-kutsussa vastaanotetaan sekä täytetty minibatch että sekvenssipituuksien vektori. Kun upotus on laskettu, muodostamme pakatun sekvenssin, välitämme sen LSTM-kerrokselle ja puramme tuloksen takaisin.

> **Huomio**: Emme itse asiassa käytä purettua tulosta `x`, koska käytämme piilotettujen kerrosten tuottamaa ulostuloa seuraavissa laskelmissa. Näin ollen voimme poistaa purkamisen kokonaan tästä koodista. Syy, miksi se on tässä, on se, että voit helposti muokata tätä koodia, jos sinun tarvitsee käyttää verkon ulostuloa myöhemmissä laskelmissa.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Huom:** Olet saattanut huomata parametrin `use_pack_sequence`, jonka välitämme koulutusfunktiolle. Tällä hetkellä `pack_padded_sequence`-funktio vaatii, että pituussekvenssin tensorin tulee olla CPU-laitteella, ja siksi koulutusfunktion täytyy välttää pituussekvenssidatan siirtämistä GPU:lle koulutuksen aikana. Voit tarkastella `train_emb`-funktion toteutusta [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py)-tiedostossa.


## Kaksisuuntaiset ja monikerroksiset RNN:t

Esimerkeissämme kaikki toistuvat verkot toimivat yhteen suuntaan, sekvenssin alusta loppuun. Tämä vaikuttaa luonnolliselta, koska se muistuttaa tapaa, jolla luemme ja kuuntelemme puhetta. Kuitenkin monissa käytännön tapauksissa meillä on satunnainen pääsy syötesekvenssiin, joten voi olla järkevää suorittaa toistuva laskenta molempiin suuntiin. Tällaisia verkkoja kutsutaan **kaksisuuntaisiksi** RNN:iksi, ja ne voidaan luoda lisäämällä `bidirectional=True` -parametri RNN/LSTM/GRU-rakentajaan.

Kun käsittelemme kaksisuuntaista verkkoa, tarvitsemme kaksi piilotilavektoria, yhden kumpaankin suuntaan. PyTorch koodaa nämä vektorit yhdeksi vektoriksi, jonka koko on kaksinkertainen, mikä on varsin kätevää, koska yleensä välität tuloksena olevan piilotilan täysin yhdistettyyn lineaariseen kerrokseen, ja sinun tarvitsee vain ottaa tämä koon kasvu huomioon kerrosta luodessasi.

Toistuva verkko, olipa se yksisuuntainen tai kaksisuuntainen, tunnistaa tiettyjä kuvioita sekvenssissä ja voi tallentaa ne tilavektoriin tai välittää ne ulostuloon. Kuten konvoluutioneuroverkoissa, voimme rakentaa toisen toistuvan kerroksen ensimmäisen päälle tunnistamaan korkeamman tason kuvioita, jotka on muodostettu ensimmäisen kerroksen tunnistamista matalan tason kuvioista. Tämä johtaa käsitteeseen **monikerroksinen RNN**, joka koostuu kahdesta tai useammasta toistuvasta verkosta, joissa edellisen kerroksen ulostulo välitetään seuraavan kerroksen syötteeksi.

![Kuva, joka esittää monikerroksista long-short-term-memory-RNN:ää](../../../../../translated_images/fi/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Kuva [tästä upeasta artikkelista](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) kirjoittanut Fernando López*

PyTorch tekee tällaisten verkkojen rakentamisesta helppoa, koska sinun tarvitsee vain lisätä `num_layers` -parametri RNN/LSTM/GRU-rakentajaan, jolloin useita toistuvia kerroksia luodaan automaattisesti. Tämä tarkoittaa myös sitä, että piilotila-/tilavektorin koko kasvaa suhteellisesti, ja sinun täytyy ottaa tämä huomioon käsitellessäsi toistuvien kerrosten ulostuloa.


## RNN:t muihin tehtäviin

Tässä osiossa olemme nähneet, että RNN:t voivat olla hyödyllisiä sekvenssiluokittelussa, mutta itse asiassa ne voivat käsitellä monia muitakin tehtäviä, kuten tekstin generointia, konekäännöstä ja paljon muuta. Tarkastelemme näitä tehtäviä seuraavassa osiossa.



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
